## 0.Data Preparation

In [4]:
import os
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import lightgbm as lgb
from lightgbm import LGBMClassifier
from bayes_opt import BayesianOptimization
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt

In [3]:
default_path = "/Users/mayritaspring/Desktop/T-Brain/Next-Premium-Prediction/"
os.chdir(default_path)

# read data
claim = pd.read_csv('../Data/claim_0702.csv')
policy = pd.read_csv('../Data/policy_0702.csv')

## 1.Data Description

In [39]:
claim.head(5)
claim.info()
claim.columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69613 entries, 0 to 69612
Data columns (total 20 columns):
Claim_Number                              69613 non-null object
Nature_of_the_claim                       69613 non-null int64
Policy_Number                             69613 non-null object
Driver's_Gender                           69613 non-null int64
Driver's_Relationship_with_Insured        69613 non-null int64
DOB_of_Driver                             69613 non-null object
Marital_Status_of_Driver                  69613 non-null int64
Accident_Date                             69613 non-null object
Cause_of_Loss                             69613 non-null object
Paid_Loss_Amount                          69613 non-null int64
paid_Expenses_Amount                      69613 non-null int64
Salvage_or_Subrogation?                   69613 non-null int64
Coverage                                  69613 non-null object
Vehicle_identifier                        69315 non-null object
At

Index(['Claim_Number', 'Nature_of_the_claim', 'Policy_Number',
       'Driver's_Gender', 'Driver's_Relationship_with_Insured',
       'DOB_of_Driver', 'Marital_Status_of_Driver', 'Accident_Date',
       'Cause_of_Loss', 'Paid_Loss_Amount', 'paid_Expenses_Amount',
       'Salvage_or_Subrogation?', 'Coverage', 'Vehicle_identifier',
       'At_Fault?', 'Claim_Status_(close,_open,_reopen_etc)', 'Deductible',
       'Accident_area', 'number_of_claimants', 'Accident_Time'],
      dtype='object')

In [6]:
policy.head(5)
policy.info()
policy.columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1747942 entries, 0 to 1747941
Data columns (total 41 columns):
Policy_Number                                 object
Insured's_ID                                  object
Prior_Policy_Number                           object
Cancellation                                  object
Vehicle_identifier                            object
Vehicle_Make_and_Model1                       object
Vehicle_Make_and_Model2                       object
Manafactured_Year_and_Month                   int64
Engine_Displacement_(Cubic_Centimeter)        int64
Imported_or_Domestic_Car                      int64
Coding_of_Vehicle_Branding_&_Type             object
qpt                                           float64
fpt                                           object
Main_Insurance_Coverage_Group                 object
Insurance_Coverage                            object
Insured_Amount1                               int64
Insured_Amount2                           

Index(['Policy_Number', 'Insured's_ID', 'Prior_Policy_Number', 'Cancellation',
       'Vehicle_identifier', 'Vehicle_Make_and_Model1',
       'Vehicle_Make_and_Model2', 'Manafactured_Year_and_Month',
       'Engine_Displacement_(Cubic_Centimeter)', 'Imported_or_Domestic_Car',
       'Coding_of_Vehicle_Branding_&_Type', 'qpt', 'fpt',
       'Main_Insurance_Coverage_Group', 'Insurance_Coverage',
       'Insured_Amount1', 'Insured_Amount2', 'Insured_Amount3',
       'Coverage_Deductible_if_applied', 'Premium',
       'Replacement_cost_of_insured_vehicle', 'Distribution_Channel',
       'Multiple_Products_with_TmNewa_(Yes_or_No?)', 'lia_class', 'plia_acc',
       'pdmg_acc', 'fassured', 'ibirth', 'fsex', 'fmarriage', 'aassured_zip',
       'iply_area', 'dbirth', 'fequipment1', 'fequipment2', 'fequipment3',
       'fequipment4', 'fequipment5', 'fequipment6', 'fequipment9',
       'nequipment9'],
      dtype='object')

## 2.Data Exploration

***
- groupby

In [11]:
# 保費小於1000者在不同保險險種下之簽單保費
policy[policy['Premium'] < 1000].groupby(by = 'Main_Insurance_Coverage_Group')[['Premium']].sum()

,Premium
Main_Insurance_Coverage_Group,
竊盜,40158719
車損,24249317
車責,352413707


***
- groupby & sum()

In [19]:
# 保費大於0者在不同保險險種下之簽單保費與保額三
policy.loc[policy['Premium'] > 0].groupby(by = 'Main_Insurance_Coverage_Group')[['Premium','Insured_Amount3']].sum()

,Premium,Insured_Amount3
Main_Insurance_Coverage_Group,,
竊盜,171001652,67009945000
車損,885384726,83348708500
車責,1262190896,9129452450000


***
- apply (max, argmax)

In [30]:
# 保費大於0者中之最大簽單保費與最大保額三
policy.loc[policy['Premium'] > 0].loc[:,['Premium','Insured_Amount3']].apply(max, axis = 0)

Premium               504932
Insured_Amount3    205000000
dtype: int64

In [63]:
# 保費大於0者中之保額一～保額三之較大column
policy.loc[policy['Premium'] > 0].loc[:,['Insured_Amount1','Insured_Amount2','Insured_Amount3']].apply(max, axis = 1)

0           10000000
1             300000
2           60000000
3             500000
4              65000
5            1000000
6             200000
7            2200000
8                184
9                 70
10                95
11              1879
12               664
13               658
14                19
15               408
16               146
17              3406
18         100000000
19            500000
20           6000000
21           3000000
22            200000
23          60000000
24            500000
25            101000
26          18000000
27            300000
28          30000000
29           2000000
             ...    
1747912      8000000
1747913      3000000
1747914       450000
1747915       450000
1747916       450000
1747917      8000000
1747918       700000
1747919      8000000
1747920      3000000
1747921       600000
1747922       100000
1747923       839000
1747924       839000
1747925       839000
1747926     36000000
1747927       600000
1747928      

In [64]:
# 保費大於0者中之保額一～保額三之較大column
policy.loc[policy['Premium'] > 0].loc[:,['Insured_Amount1','Insured_Amount2','Insured_Amount3']].apply(np.argmax, axis = 1)

0          Insured_Amount3
1          Insured_Amount3
2          Insured_Amount3
3          Insured_Amount3
4          Insured_Amount3
5          Insured_Amount3
6          Insured_Amount3
7          Insured_Amount3
8          Insured_Amount1
9          Insured_Amount1
10         Insured_Amount1
11         Insured_Amount1
12         Insured_Amount1
13         Insured_Amount1
14         Insured_Amount1
15         Insured_Amount1
16         Insured_Amount1
17         Insured_Amount1
18         Insured_Amount3
19         Insured_Amount3
20         Insured_Amount3
21         Insured_Amount3
22         Insured_Amount3
23         Insured_Amount3
24         Insured_Amount3
25         Insured_Amount3
26         Insured_Amount3
27         Insured_Amount3
28         Insured_Amount3
29         Insured_Amount3
                ...       
1747912    Insured_Amount3
1747913    Insured_Amount3
1747914    Insured_Amount3
1747915    Insured_Amount3
1747916    Insured_Amount3
1747917    Insured_Amount3
1

***
- create column & apply

In [33]:
#創欄位：重置成本之無條件進入取至整數位
policy['Replacement_cost_of_insured_vehicle_ceil'] = policy.Replacement_cost_of_insured_vehicle.apply(np.ceil)
policy.loc[0:4,['Replacement_cost_of_insured_vehicle', 'Replacement_cost_of_insured_vehicle_ceil']]

,Replacement_cost_of_insured_vehicle,Replacement_cost_of_insured_vehicle_ceil
0,45.4,46.0
1,45.4,46.0
2,59.9,60.0
3,59.9,60.0
4,39.9,40.0


***
- lambda

In [58]:
#創欄位：出險月份
claim['Accident_Month'] = claim.Accident_Date.str.split('/').apply(lambda x: x[1])
claim.loc[:,['Accident_Date', 'Accident_Month']].head()

,Accident_Date,Accident_Month
0,2015/01,01
1,2015/01,01
2,2015/01,01
3,2015/01,01
4,2015/01,01


In [59]:
claim.Accident_Month.unique()

array(['01', '02', '03', '06', '04', '08', '05', '07', '09', '10', '11',
       '12'], dtype=object)

***
- applymap

In [77]:
#針對選取出的表格，對每個cel取float
#排氣量大於1000者，將其排氣量改為float
policy.loc[policy['Engine_Displacement_(Cubic_Centimeter)'] > 1000].loc[:,['Engine_Displacement_(Cubic_Centimeter)']].applymap(float).head()

,Engine_Displacement_(Cubic_Centimeter)
0,1342.0
1,1342.0
2,1794.0
3,1794.0
4,1341.0


In [78]:
format = lambda x: '%.2f' % x
policy.loc[policy['Engine_Displacement_(Cubic_Centimeter)'] > 1000].loc[:,['Engine_Displacement_(Cubic_Centimeter)']].applymap(format).head()

,Engine_Displacement_(Cubic_Centimeter)
0,1342.00
1,1342.00
2,1794.00
3,1794.00
4,1341.00


In [82]:
#利用map定義字典格式，以作轉換
#將車系代號轉為中文類別
policy['Imported_or_Domestic_Car_Cate'] =  policy.Imported_or_Domestic_Car.map({10:'國產車',20:'美國車',
21:'福特', 22:'通用', 23:'克萊斯勒', 24:'美規日車',30:'歐洲車',40:'日本車',50:'韓國車',90:'其它'})

In [83]:
policy.loc[0:4,['Imported_or_Domestic_Car','Imported_or_Domestic_Car_Cate']]

,Imported_or_Domestic_Car,Imported_or_Domestic_Car_Cate
0,10,國產車
1,10,國產車
2,10,國產車
3,10,國產車
4,10,國產車
